In [18]:
import pandas as pd
import numpy as np
!pip install wordcloud
import warnings
warnings.filterwarnings("ignore")                     
import nltk                                         
from nltk.corpus import stopwords                                   
import re
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer          
# from sklearn.feature_extraction.text import TfidfVectorizer          
from gensim.models import Word2Vec                                   
from tensorflow.keras.layers import Embedding
# from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
# from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score, mean_absolute_error, accuracy_score
from keras.callbacks import History
from tensorflow.keras.layers import Reshape

[nltk_data] Downloading package stopwords to C:\Users\Srija
[nltk_data]     Kandula\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [49]:
csv_path = 'dataset/training_set_rel3.tsv'
# Reading the training data csv file
training_data = pd.read_csv(csv_path, sep='\t', encoding='latin1')
csv_path1 = 'dataset/test_set.tsv'
# Reading the testing data csv file
testing_data = pd.read_csv(csv_path1, sep='\t', encoding='latin1')
training_data = training_data[['essay_id','essay','domain1_score']]

# testing_data.head()
training_data = training_data[training_data['domain1_score'] <= 10]
training_data.count()
training_data.head(5)

,essay_id,essay,domain1_score
0,1,"Dear local newspaper, I think effects computer...",8
1,2,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,"Dear @LOCATION1, I know having computers has a...",8


In [20]:
# Preprocess essays and tokenize into words
def preprocess_essay(essay):
    essay = re.sub('[^a-zA-Z]', ' ', essay)
    essay = essay.lower()
    words = word_tokenize(essay)
    stops = set(stopwords.words('english'))
    words = [word for word in words if word not in stops]
    return words

# preprocessing each essay in the DataFrame
training_data['processed_essay'] = training_data['essay'].apply(preprocess_essay)
testing_data['processed_essay'] = testing_data['essay'].apply(preprocess_essay)


In [21]:
# Training the Word2Vec model
word2vec_model = Word2Vec(sentences=training_data['processed_essay'], vector_size=300, window=5, min_count=1, workers=4)

# Converting essays into numerical feature vectors using Word2Vec embeddings
def get_sentence_vector(sentence, model):
    vector_sum = 0
    for word in sentence:
        if word in model.wv:
            vector_sum += model.wv[word]
    return vector_sum.tolist() if isinstance(vector_sum, np.ndarray) else [np.zeros(model.vector_size)]

word2vec_model = Word2Vec(sentences=training_data['processed_essay'], vector_size=300, window=5, min_count=1, workers=4)

training_data['word_vectors'] = training_data['processed_essay'].apply(lambda x: get_sentence_vector(x, word2vec_model))
testing_data['word_vectors'] = testing_data['processed_essay'].apply(lambda x: get_sentence_vector(x, word2vec_model))

In [22]:
X_train = np.array(training_data['word_vectors'].tolist())

y_train = np.array(training_data['domain1_score'])


In [23]:
def get_Lstm(input_shape):
    model = Sequential()
    model.add(Reshape((1, input_shape[0]), input_shape=(input_shape[0],)))
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, return_sequences=True))
    model.add(LSTM(200, recurrent_dropout=0.4, return_sequences=True))
    model.add(LSTM(100, recurrent_dropout=0.4, return_sequences=True)) 
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_absolute_error', optimizer='rmsprop', metrics=['mae'])
    
    model.summary()
    return model

In [24]:
# Build the LSTM model
model = get_Lstm(input_shape=(X_train.shape[1],))

model.fit(X_train, y_train, epochs=10, batch_size=64)

# Preprocessing the essays in the testing data
testing_data['processed_essay'] = testing_data['essay'].apply(preprocess_essay)

# Converting the testing data essays into numerical feature vectors using Word2Vec embeddings
testing_data['word_vectors'] = testing_data['processed_essay'].apply(lambda x: get_sentence_vector(x, word2vec_model))

# Converting the Word vectors into numpy arrays for testing
X_test = np.array(testing_data['word_vectors'].tolist())
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))

# Predicting scores on the testing data
y_pred = model.predict(X_test)

#predicted scores
testing_data['predicted_score'] = y_pred
print(testing_data[['predicted_score']])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_3 (Reshape)         (None, 1, 300)            0         
                                                                 
 lstm_12 (LSTM)              (None, 1, 300)            721200    
                                                                 
 lstm_13 (LSTM)              (None, 1, 200)            400800    
                                                                 
 lstm_14 (LSTM)              (None, 1, 100)            120400    
                                                                 
 lstm_15 (LSTM)              (None, 64)                42240     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                

In [33]:
# output_csv_path = 'predicted.csv'

# # Save the DataFrame to a CSV file
# testing_data.to_csv(output_csv_path, index=False)

In [43]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error, cohen_kappa_score, accuracy_score
from keras.callbacks import History

true_scores = training_data['domain1_score']
history = History()

bins = [0, 3, 6, 8, 10]
labels = ['1-3', '4-6', '7-8', '9-10']

# number of folds
num_folds = 3
kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# lists to store results from each fold
mae_scores = []
kappa_scores = []
accuracy_scores = []

# Iterating each folds
for train_index, test_index in kf.split(training_data['word_vectors'], training_data['domain1_score']):
    X_train, X_val = np.array(training_data['word_vectors'].tolist())[train_index], np.array(training_data['word_vectors'].tolist())[test_index]
    y_train, y_val = true_scores.iloc[train_index], true_scores.iloc[test_index]

    # input shape
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

    # Train LSTM model
    lstm_model = get_Lstm(input_shape=(X_train.shape[1], 1))
    lstm_model.fit(X_train, y_train, batch_size=64, epochs=25, callbacks=[history])

    # Predicting on the validation set
    y_pred_val = lstm_model.predict(X_val)
    y_pred_val_rounded = np.around(y_pred_val)

    # Converting true scores to categories
    true_categories_val = pd.cut(y_val, bins=bins, labels=labels, include_lowest=True).astype(str)

    # Converting predicted scores to categories
    predicted_categories_val = pd.cut(y_pred_val_rounded.flatten(), bins=bins, labels=labels, include_lowest=True).astype(str)

    # Calculating  metrics for the validation set
    mae_val = mean_absolute_error(y_val, y_pred_val_rounded)
    kappa_val = cohen_kappa_score(true_categories_val, predicted_categories_val, weights='quadratic')
    accuracy_val = accuracy_score(true_categories_val, predicted_categories_val)

    # Storing the results
    mae_scores.append(mae_val)
    kappa_scores.append(kappa_val)
    accuracy_scores.append(accuracy_val)

# Calculating the average scores over all folds
avg_mae = np.mean(mae_scores)
avg_kappa = np.mean(kappa_scores)
avg_accuracy = np.mean(accuracy_scores)

print("Average MAE across folds: {}".format(avg_mae))
print("Average Kappa Score across folds: {}".format(avg_kappa))
print("Average Accuracy across folds: {}".format(avg_accuracy))


Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_18 (Reshape)        (None, 1, 300)            0         
                                                                 
 lstm_72 (LSTM)              (None, 1, 300)            721200    
                                                                 
 lstm_73 (LSTM)              (None, 1, 200)            400800    
                                                                 
 lstm_74 (LSTM)              (None, 1, 100)            120400    
                                                                 
 lstm_75 (LSTM)              (None, 64)                42240     
                                                                 
 dropout_18 (Dropout)        (None, 64)                0         
                                                                 
 dense_18 (Dense)            (None, 1)               

                                                                 
 lstm_82 (LSTM)              (None, 1, 100)            120400    
                                                                 
 lstm_83 (LSTM)              (None, 64)                42240     
                                                                 
 dropout_20 (Dropout)        (None, 64)                0         
                                                                 
 dense_20 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1284705 (4.90 MB)
Trainable params: 1284705 (4.90 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/25
112/112 [==============================] - 20s 29ms/step - loss: 1.5329 - mae: 1.5329
Epoch 2/25
112/112 [==============================] - 3s 28ms/step - loss: 1.0312 - mae: 1.0312
Epoch 3/25
112/112 [=======================

In [47]:
import matplotlib.pyplot as plt
# Merging training and testing data on 'essay_id'
merged_data = pd.merge(training_data[['essay_id', 'domain1_score']], testing_data[['essay_id', 'predicted_score']], on='essay_id', how='inner')

# Creating scatter plot
plt.scatter(merged_data['domain1_score'], merged_data['predicted_score'])
plt.title('Actual vs. Predicted Scores based on Essay ID')
plt.xlabel('Actual Scores (Training Data)')
plt.ylabel('Predicted Scores (Testing Data)')
plt.show()


In [ ]:
plt.hist(merged_data['domain1_score'], bins=20, alpha=0.5, label='Actual Scores (Training Data)')
plt.hist(merged_data['predicted_score'], bins=20, alpha=0.5, label='Predicted Scores (Testing Data)')
plt.title('Histogram of Actual and Predicted Scores based on Essay ID')
plt.xlabel('actual')
plt.ylabel('predicted')
plt.legend()
plt.show()